# ライフゲーム

ここでは、代表的な2次元セルオートマトンである[ライフゲーム](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%82%A4%E3%83%95%E3%82%B2%E3%83%BC%E3%83%A0)（Conway's Game of Life）の、Pythonでの実装の1つを与える。

高速化のため[Numba](https://numba.pydata.org/)を用いる<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)。Google ColaboratoryやAnacondaを使っている場合はすでにインストール済みのはずである。Binderの場合は、次のセルを実行してNumbaをインストールする必要がある。

In [ ]:
! [ -n "$BINDER_SERVICE_HOST" ] && pip install numba

以下が（周期境界条件を用いた）ライフゲームの実装である。この中でもっとも重要な関数は`evolve`であり、全セルの状態を1ステップ進める計算をしている。実際にライフゲームを実行するには`game_of_life`関数を使う<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import numba
from matplotlib.animation import FuncAnimation
import urllib.request
import functools
import re


def empty_cells(width, height):
    """`(width, height)`のサイズの、すべて0のセルを作成して返す。"""
    return np.zeros((height, width), dtype="int8")


def randomized_cells(width, height, margin=0):
    """`(width, height)`のサイズの、ランダムなセルを作成して返す。"""
    cells = np.random.randint(2, size=(height, width), dtype="int8")
    if margin > 0:
        cells = np.pad(cells, ((margin, margin), (margin, margin)))
    return cells


@functools.lru_cache(maxsize=None)
def read_from_url(url):
    """指定されたURLからテキストデータを読み込む。"""
    return urllib.request.urlopen(url).read().decode("utf-8")


def ensure_cells_size(cells, new_width, new_height):
    """指定されたサイズ以上の大きさのセルを確保する。"""
    new_width = max(new_width, 1)
    new_height = max(new_height, 1)

    if cells is None:
        return empty_cells(new_width, new_height)

    old_height, old_width = cells.shape

    if new_width <= old_width and new_height <= old_height:
        return cells

    new_width = max(new_width, old_width)
    new_height = max(new_height, old_height)

    return np.pad(cells, ((0, new_height - old_height), (0, new_width - old_width)))


def read_cells_from_plaintext(string):
    """plaintext形式（とLife 1.05形式）のセルを読みこむ。"""
    # 「!」と「#」で始まるコメント行を除去。空行も除去。
    lines = string.splitlines()
    lines = [line for line in lines if not re.match(r"^\s*[!#]", line)]
    lines = [line.rstrip() for line in lines]
    lines = [line for line in lines if line]

    width = max(len(line) for line in lines)
    height = len(lines)

    width = max(width, 1)
    height = max(height, 1)

    cells = empty_cells(width, height)

    for i, line in enumerate(lines):
        for j, c in enumerate(line):
            if c in "Oo*":
                cells[i, j] = 1
            elif c in ". ":
                pass
            else:
                raise ValueError(f"unexpected character: {line}:{c}")

    return cells


def read_cells_from_rle(string):
    """RLE形式のセルを読みこむ。"""
    cells = None
    x = 0
    y = 0

    # 「#」で始まるコメントを除去。
    lines = string.splitlines()
    lines = [re.sub("#.*", "", line).strip() for line in lines]
    lines = [line for line in lines if line]

    # RLEを1つの文字列に戻す。

    string = ""

    for line in lines:
        # ヘッダー「x = m, y = n ...」をチェック。
        m = re.match(r"^\s*x\s*=\s*(\d+)\s*,\s*y\s*=\s*(\d+)", line)
        if m:
            cells = ensure_cells_size(cells, int(m.group(1)), int(m.group(2)))
            continue
        string += line

    # 終端記号「!」以降と空白を除去。
    string = re.sub("!.*", "", string)
    string = re.sub("\s+", "", string)

    # 少なくとも 1 x 1 のセルを確保。
    cells = ensure_cells_size(cells, 1, 1)

    # RLE中の各アイテムを処理。
    for s in re.findall(r"\d+\D|\D", string):
        if len(s) >= 2:
            n = int(s[:-1])
            c = s[-1]
        else:
            n = 1
            c = s[0]
        if c == "b":
            h, _ = cells.shape
            cells = ensure_cells_size(cells, x + n, h)
            x += n
        elif c == "o":
            h, _ = cells.shape
            cells = ensure_cells_size(cells, x + n, h)
            for i in range(n):
                cells[y, x + i] = 1
            x += n
        elif c == "$":
            _, w = cells.shape
            cells = ensure_cells_size(cells, w, y + n + 1)
            x = 0
            y += n
        else:
            raise ValueError(f"unexpected item: {s}")

    return cells


def guess_pattern_format(string):
    """パターンファイルの形式を推測する。"""
    lines = string.splitlines()
    # 「!」と「#」で始まるコメントを除去。
    lines = [re.sub("!.*", "", line).strip() for line in lines]
    lines = [re.sub("#.*", "", line).strip() for line in lines]

    # 最初に見つかった文字で推測。
    for c in "".join(lines):
        if c == ".":
            return "plaintext"
        elif c in "b$":
            return "rle"

    # 推測不能。
    return None


def load_cells(
    string, fmt=None, cells=None, width=None, height=None, x=None, y=None, margin=None
):
    """セルを読み込んで返す。"""
    # URLらしき場合はそこからテキストを読み込む。

    if string.startswith("http"):
        url_lowered = string.lower()
        string = read_from_url(string)
        if fmt is None:
            if any(url_lowered.endswith(s) for s in (".cells", ".lif", ".life")):
                fmt = "plaintext"
            elif url_lowered.endswith(".rle"):
                fmt = "rle"

    # セルの読み込み。

    if fmt is None:
        fmt = guess_pattern_format(string)

    if fmt == "plaintext":
        data = read_cells_from_plaintext(string)
    elif fmt == "rle":
        data = read_cells_from_rle(string)
    else:
        raise ValueError("please specify fmt")

    # 結果の幅と高さ。
    # 安定パターンが周期的境界条件で死なないようにマージンを設ける。
    # +1では足りないパターンファイルがあるので多めに。
    current_height, current_width = data.shape
    if margin is None:
        margin = 5
    new_width = current_width + margin * 2
    if width:
        new_width = max(new_width, width)
    new_height = current_height + margin * 2
    if height:
        new_height = max(new_height, height)

    if cells is None:
        # 背景セルなし。
        cells = empty_cells(new_width, new_height)
    else:
        # 背景セルあり。
        cells = ensure_cells_size(np.atleast_2d(cells).copy(), new_width, new_height)

    # 位置の決定。
    if x is None:
        x = (new_width - current_width) // 2
    if y is None:
        y = (new_height - current_height) // 2

    # 貼り付け。
    for i in range(current_height):
        for j in range(current_width):
            if data[i, j]:
                cells[(i + y) % new_height, (j + x) % new_width] = 1

    return cells


@numba.jit(nopython=True)
def evolve(cells, new_cells, rule):
    """`cells`を`rule`を使って1ステップ進めたものを`new_cells`に書き出す。"""
    nrows, ncols = cells.shape
    for i in range(nrows):
        for j in range(ncols):
            nlives = 0
            nlives += cells[(i - 1) % nrows, (j - 1) % ncols]
            nlives += cells[(i - 1) % nrows, j]
            nlives += cells[(i - 1) % nrows, (j + 1) % ncols]
            nlives += cells[i, (j - 1) % ncols]
            nlives += cells[i, (j + 1) % ncols]
            nlives += cells[(i + 1) % nrows, (j - 1) % ncols]
            nlives += cells[(i + 1) % nrows, j]
            nlives += cells[(i + 1) % nrows, (j + 1) % ncols]
            new_cells[i, j] = rule[cells[i, j], nlives]


def game_of_life(
    initial_cells=randomized_cells,
    rule=None,
    width=200,
    height=200,
    tmax=1000,
    cell_size=None,
    cmap="viridis",
    dpi=72,
    interval=50,
):
    """ライフゲームを実行して結果を表示する。

    Parameters
    ----------
    initial_cells: array_like or callable, optional
        初期状態、あるいは初期状態を生成する関数。
    rule: array_like, optional
        ルールを与える2 x 8の配列。
    width: int, optional
        初期状態の幅。initial_cellsを関数として呼び出すときに使われる。
    height: int, optional
        初期状態の高さ。initial_cellsを関数として呼び出すときに使われる。
    tmax: int, optional
        最終時刻。
    cell_size: int, optional
        描画時の各セルの大きさ。
    cmap: str or Colormap, optional
        描画に使うカラーマップ。
    dpi: int, optional
        描画時のDPI。
    interval: int, optional
        アニメーションの各フレーム間のインターバル。

    Returns
    -------
    None

    """
    if callable(initial_cells):
        # 関数であればこれを呼び出して初期状態を決める。
        initial_cells = initial_cells(width, height)

    if rule is None:
        # デフォルト: 標準的なB3/S23のルール。
        rule = ((0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 1, 1, 0, 0, 0, 0, 0))
    rule = np.atleast_2d(rule)

    # セルを用意する。
    cells = np.copy(np.atleast_2d(initial_cells))
    new_cells = np.empty_like(cells)

    # アニメーションを作成する。

    nrows, ncols = cells.shape

    if cell_size is None:
        cell_count = max(nrows, ncols)
        cell_size = max(int(500 // cell_count), 1)

    fig, ax = plt.subplots(
        figsize=(cell_size * ncols / dpi, cell_size * nrows / dpi), dpi=dpi
    )
    matshow = ax.matshow(cells, vmin=0, vmax=1, cmap=cmap)
    ax.axis("off")
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    def animate(t):
        nonlocal cells, new_cells
        if t > 0:
            # 1ステップ進める。
            evolve(cells, new_cells, rule)
            cells, new_cells = new_cells, cells
        matshow.set_data(cells)
        ax.set_title(
            f"$t = {t}$",
            fontsize=18,
            x=0.1,
            y=0.9,
            horizontalalignment="left",
            verticalalignment="top",
            color="white",
            backgroundcolor=(0, 0, 0, 0.5),
        )

    anim = FuncAnimation(fig, animate, frames=tmax + 1, interval=interval)
    html = display.HTML(anim.to_jshtml())
    display.display(html)
    plt.close()

次のセルでは、$200\times 200$のセルを用いて、ランダムな初期状態から$t=1000$までライフゲームを実行し、アニメーションを生成する。実行に1分程度かかり、ノートブックに保存されるデータは10MB超の大きさになるので注意すること。

In [ ]:
%%time
game_of_life()

次のセルでは、[LifeWiki](https://conwaylife.com/wiki/Main_Page)に公開されている[RLE](https://conwaylife.com/wiki/Run_Length_Encoded)形式のファイルに保存されたパターンを読み込んで、[グライダー](https://conwaylife.com/wiki/Glider)を発生させている。

In [ ]:
%%time
cells = load_cells(
    "https://www.conwaylife.com/patterns/glider.rle", width=50, height=50
)
game_of_life(cells, tmax=199)

インターネットで気になるRLE形式のパターンファイルを見つけたら、上の例のように走らせてみるとよい。次のセルのパターンは[ロビン卿](https://conwaylife.com/wiki/Sir_Robin)（Sir Robin）と呼ばれる。

In [ ]:
%%time
cells = load_cells("https://www.conwaylife.com/patterns/sirrobin.rle", margin=30)
game_of_life(cells, tmax=150)

次のセルのパターンは[ゴスパーのグライダー銃](https://conwaylife.com/wiki/Gosper_glider_gun)（Gosper glider gun）と呼ばれる。

In [ ]:
%%time
cells = load_cells("https://conwaylife.com/patterns/gosperglidergun.rle", margin=30)
game_of_life(cells, tmax=500)

次のセルのパターンは[ダブルX](https://conwaylife.com/wiki/Double_X)（Double X）と呼ばれる。

In [ ]:
%%time
cells = load_cells("https://conwaylife.com/patterns/doublex.rle")
game_of_life(cells, tmax=45)

<!-- textlint-disable
ja-engineering-paper/use-si-units
-->

次のセルでは2022年6月14日に発見された[周期10のドミノ噴水](https://conwaylife.com/wiki/P10_domino_fountain)（P10 domino fountain）を発生させている。RLEファイルへのリンクがまだないようなので、RLEファイルの内容をそのまま`load_cells`関数に渡している。

<!-- textlint-enable -->

In [ ]:
%%time
cells = load_cells(
    """
x = 34, y = 21, rule = B3/S23
16b2o2$13bo6bo$12bo8bo$13b8o3$8b2o14b2o$2b2o4b6o6b6o4b2o$2b3o5b5o4b5o
5b3o$b2o2b2o2bo3bobo2bobo3bo2b2o2b2o$o2b2obob3obo2b4o2bob3obob2o2bo$2o
bo2bobo2bobo6bobo2bobo2bob2o$3bob2ob2obob8obob2ob2obo$2obo2bo2bobo10bo
bo2bo2bob2o$2ob2o2bobobob8obobobo2b2ob2o$8bo3bo8bo3bo$14b6o$15bo2bo$
13bo6bo$13b2o4b2o!
"""
)
game_of_life(cells, tmax=9)

もしパターンを手で入力するなら[Plaintext形式](https://conwaylife.com/wiki/Plaintext)が便利である。次のセルでは、[サンダーバード](https://conwaylife.com/wiki/Thunderbird)と呼ばれるパターンをPlaintext形式でそのまま`load_cells`関数に渡している。

In [ ]:
%%time
cells = load_cells(
    """
ooo
...
.o.
.o.
.o.
""",
    margin=30,
)
game_of_life(cells, tmax=300)

ライフゲームのルールを変化させることもできる。次のセルでは[mazectric](https://conwaylife.com/wiki/Mazectric)（B3/S1234）と呼ばれるルールで実行している。

In [ ]:
%%time
cells = randomized_cells(50, 50, margin=75)
game_of_life(cells, rule=[[0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0]], tmax=350)

## 脚注

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
`evolve`関数の処理の高速化の結果、現在のところ圧倒的に遅いのはシミュレーション自体ではなくアニメーションを作成している部分である。

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
このコードの多くの部分（`read_from_url`関数から`load_cells`関数まで）はパターンファイルを読み込む処理のためのものであり、ライフゲームをランダムな初期条件から開始するのであれば必要ない。